In [1]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 6.1MB/s 
     |████████████████████████████████| 2.1MB 22.1MB/s 
     |████████████████████████████████| 1.2MB 39.1MB/s 
     |████████████████████████████████| 870kB 38.0MB/s 
     |████████████████████████████████| 3.3MB 43.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=d14b8728047957e2bf1880fee662ba6f36af1028861de800701570927056b16c
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=5578efac018174b076bc5b7ab686820611252097219c100a26c49fb61b6381c9
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [ ]:
# import necessary files
import pandas as pd
# import seaborn as sns
import numpy as np
# import spacy
# import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report, precision_recall_curve, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.feature_extraction.text import TfidfVectorizer
import time
import re
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import nltk
nltk.download('stopwords')
from sklearn.metrics import roc_auc_score
from keras.preprocessing import text, sequence
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
import pickle
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data cleaning method: removes http links, special characters and user tagged comments 
def clean_data(text):
  text = text.map(lambda x: re.sub('\\n',' ',str(x)))
  text = text.map(lambda x: re.sub("\[\[User.*",'',str(x)))
  text = text.map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
  text = text.map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
  return text

stopset = set(stopwords.words('english'))
stopset.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

# Method to remove stopwords
def remove_stopword(text):
  tokenizer = RegexpTokenizer(r'\w+')
  processed = []
  for comment in tqdm(text):
    tokens = tokenizer.tokenize(comment)
    word_list = [word for word in tokens if word not in stopset]
    processed.append(" ".join(word_list))
  return processed

To run this notebook, place the data folder into your Google Drive and change the data path below (line 2) to your data directory.

In [ ]:
# read train and test data
data_path = "/content/drive/My Drive/NLP_Data/"
train = pd.read_csv(data_path+"train.csv")
train = train.fillna('na')
test = pd.read_csv(data_path+"test.csv")
test = test.fillna('na')
test_labels = pd.read_csv(data_path+"test_labels.csv")
# print(train.columns)

#remove rows from test set that have -1s
test_with_labels = pd.concat([test, test_labels], axis = 1)
test_with_labels = test_with_labels[test_with_labels['toxic'] != -1]

#remove unnecessary columns
X_train = train['comment_text']
X_test = test_with_labels['comment_text']
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']]
y_test = test_with_labels[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']]

# Cleaning the df 
clean_train = clean_data(X_train).tolist()
clean_test = clean_data(X_test).tolist()

# Removing stop words 
clean_train = remove_stopword(clean_train)
clean_test = remove_stopword(clean_test)

X_train1 = pd.DataFrame(clean_train, columns=['comment_text'])
X_test1 = pd.DataFrame(clean_test, columns=['comment_text'])

# creating a subset of each dataset in order to run relatively fast with BERT
X_train_subset = X_train1[:2000]
X_test_subset = X_test1[:400]
y_train_subset = y_train[:2000]
y_test_subset = y_test[:400]

# Splitting training data into train and validation sets 
X_train, X_val, y_train, y_val = train_test_split(X_train_subset, y_train_subset, test_size=0.2, random_state=42)

# Printing lengths of train, test and validation to split after vectorization 
print("length of train is ..." + str(len(X_train)))
print("length of validation is ..." + str(len(X_val)))
print("length of test is ..." + str(len(X_test_subset)))

total time 4.5119030475616455


  2%|▏         | 3940/159571 [00:00<00:03, 39396.94it/s]

total time 1.79317307472229


100%|██████████| 63978/63978 [00:01<00:00, 39025.72it/s]

length of train is ...1600
length of validation is ...400
length of test is ...400


In [ ]:
print(X_train_subset.head())

                                        comment_text
0  Explanation Why edits made username Hardcore M...
1  D aww He matches background colour I seemingly...
2  Hey man I really trying edit war It guy consta...
3  More I make real suggestions improvement I won...
4              You sir hero Any chance remember page


In [ ]:
# Function for vectorization using BERT
def bert_vectorize(df, model):
    start_time=time.time()
    sentences = df['comment_text'].tolist()
    #load pretrained BERT model
    #encode sentences
    vectors = model.encode(sentences)
    end_time=time.time()
    print("total time to vectorize: ",end_time-start_time)
    return list(vectors)

In [ ]:
# Vectorize data using BERT
model = SentenceTransformer('bert-base-nli-stsb-mean-tokens')

vectorized_X_train_subset = bert_vectorize(X_train, model)
vectorized_X_val = bert_vectorize(X_val, model)
vectorized_X_test = bert_vectorize(X_test_subset, model)

# verifying lengths of train, test and validation after vectorization 
print("length of train is ..." + str(len(vectorized_X_train_subset)))
print("length of validation is ..." + str(len(vectorized_X_val)))
print("length of test is ..." + str(len(X_test_subset)))

100%|██████████| 405M/405M [00:18<00:00, 21.7MB/s]


total time to vectorize:  251.22597742080688
total time to vectorize:  64.99932622909546
total time to vectorize:  85.3821349143982
length of train is ...1600
length of validation is ...400
length of test is ...400


In [ ]:
# classify data: LogReg
def logreg(X_train, y_train, X_test, y_test):
  text_classifier = LogisticRegression(solver='sag', C=10)
  lr_pipeline = Pipeline([('clf', OneVsRestClassifier(text_classifier)),])
  categories = list(y_train.columns.values)
  print(categories)
  y_pred = []
  for category in categories:
      print(category)
      
      # Training logistic regression model on train data
      lr_pipeline.fit(X_train, y_train[category])
      
      # calculating test accuracy
      predictions = lr_pipeline.predict(X_test)
      y_pred.append(predictions)
      print('Test accuracy is {}'.format(accuracy_score(y_true=y_test[category], y_pred=predictions)))
      print("\n")

      print('Test F1 score is {}'.format(f1_score(y_true=y_test[category], y_pred=predictions, average='weighted')))
      print('\n')

      print('classification report per label')
      print(classification_report(y_pred=predictions, y_true=y_test[category]))

  return predictions

In [ ]:
# classify validation data 
val_pred = logreg(vectorized_X_train_subset, y_train, vectorized_X_val, y_val)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
**Processing toxic comments...**


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Test accuracy is 0.91


Test F1 score is 0.9033798882681563


classification report per label
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       351
           1       0.69      0.49      0.57        49

    accuracy                           0.91       400
   macro avg       0.81      0.73      0.76       400
weighted avg       0.90      0.91      0.90       400

**Processing severe_toxic comments...**


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Test accuracy is 0.9775


Test F1 score is 0.9744745938472174


classification report per label
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       393
           1       0.25      0.14      0.18         7

    accuracy                           0.98       400
   macro avg       0.62      0.57      0.59       400
weighted avg       0.97      0.98      0.97       400

**Processing obscene comments...**


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Test accuracy is 1.0


Test F1 score is 1.0


classification report per label
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       399
           1       1.00      1.00      1.00         1

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

**Processing insult comments...**


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Test accuracy is 0.955


Test F1 score is 0.9541600000000001


classification report per label
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       374
           1       0.67      0.62      0.64        26

    accuracy                           0.95       400
   macro avg       0.82      0.80      0.81       400
weighted avg       0.95      0.95      0.95       400

**Processing identity_hate comments...**
Test accuracy is 0.9875


Test F1 score is 0.984845072959827


classification report per label
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       395
           1       0.50      0.20      0.29         5

    accuracy                           0.99       400
   macro avg       0.74      0.60      0.64       400
weighted avg       0.98      0.99      0.98       400



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
# classify test data
# this time don't split X_train into train and val sets
vectorized_X_train = bert_vectorize(X_train_subset, model)

total time to vectorize:  311.8511230945587


In [ ]:
y_pred = logreg(vectorized_X_train, y_train_subset, vectorized_X_test, y_test_subset)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
**Processing toxic comments...**


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Test accuracy is 0.9025


Test F1 score is 0.9118474633650916


classification report per label
              precision    recall  f1-score   support

           0       0.97      0.92      0.95       366
           1       0.45      0.71      0.55        34

    accuracy                           0.90       400
   macro avg       0.71      0.81      0.75       400
weighted avg       0.93      0.90      0.91       400

**Processing severe_toxic comments...**


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Test accuracy is 0.9925


Test F1 score is 0.9937452948557088


classification report per label
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       399
           1       0.00      0.00      0.00         1

    accuracy                           0.99       400
   macro avg       0.50      0.50      0.50       400
weighted avg       0.99      0.99      0.99       400

**Processing obscene comments...**


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Test accuracy is 0.995


Test F1 score is 0.995


classification report per label
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       399
           1       0.00      0.00      0.00         1

    accuracy                           0.99       400
   macro avg       0.50      0.50      0.50       400
weighted avg       0.99      0.99      0.99       400

**Processing insult comments...**


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Test accuracy is 0.94


Test F1 score is 0.9425685425685425


classification report per label
              precision    recall  f1-score   support

           0       0.97      0.96      0.97       380
           1       0.42      0.50      0.45        20

    accuracy                           0.94       400
   macro avg       0.70      0.73      0.71       400
weighted avg       0.95      0.94      0.94       400

**Processing identity_hate comments...**
Test accuracy is 0.985


Test F1 score is 0.9800377833753149


classification report per label
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       395
           1       0.00      0.00      0.00         5

    accuracy                           0.98       400
   macro avg       0.49      0.50      0.50       400
weighted avg       0.98      0.98      0.98       400



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [ ]:
# convert y_pred to df with each category's predictions getting slotted into the corresponding column
y_pred_df = pd.DataFrame(columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
y_pred_df["toxic"] = y_pred[0]
y_pred_df["severe_toxic"] = y_pred[1]
y_pred_df["obscene"] = y_pred[2]
y_pred_df["threat"] = y_pred[3]
y_pred_df["insult"] = y_pred[4]
y_pred_df["identity_hate"] = y_pred[5]
print(y_pred_df)

     toxic  severe_toxic  obscene  threat  insult  identity_hate
0        0             0        0       0       0              0
1        1             0        0       0       0              0
2        0             0        0       0       0              0
3        0             0        0       0       0              0
4        0             0        0       0       0              0
..     ...           ...      ...     ...     ...            ...
395      0             0        0       0       0              0
396      0             0        0       0       0              0
397      0             0        0       0       0              0
398      0             0        0       0       0              0
399      0             0        0       0       0              0

[400 rows x 6 columns]


In [ ]:
print(classification_report(y_pred=y_pred_df['toxic'], y_true=y_test_subset['toxic']))
print(classification_report(y_pred=y_pred_df['severe_toxic'], y_true=y_test_subset['severe_toxic']))
print(classification_report(y_pred=y_pred_df['obscene'], y_true=y_test_subset['obscene']))
print(classification_report(y_pred=y_pred_df['threat'], y_true=y_test_subset['threat']))
print(classification_report(y_pred=y_pred_df['insult'], y_true=y_test_subset['insult']))
print(classification_report(y_pred=y_pred_df['identity_hate'], y_true=y_test_subset['identity_hate']))

              precision    recall  f1-score   support

           0       0.97      0.92      0.95       366
           1       0.45      0.71      0.55        34

    accuracy                           0.90       400
   macro avg       0.71      0.81      0.75       400
weighted avg       0.93      0.90      0.91       400

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       399
           1       0.00      0.00      0.00         1

    accuracy                           0.99       400
   macro avg       0.50      0.50      0.50       400
weighted avg       0.99      0.99      0.99       400

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       378
           1       0.40      0.27      0.32        22

    accuracy                           0.94       400
   macro avg       0.68      0.62      0.65       400
weighted avg       0.93      0.94      0.93       400

              preci

In [ ]:
print(y_test_subset)
print(roc_auc_score(y_true=y_test_subset[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values, y_score=y_pred_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values))

     toxic  severe_toxic  obscene  threat  insult  identity_hate
5        0             0        0       0       0              0
7        0             0        0       0       0              0
11       0             0        0       0       0              0
13       0             0        0       0       0              0
14       0             0        0       0       0              0
..     ...           ...      ...     ...     ...            ...
931      0             0        0       0       0              0
934      0             0        0       0       0              0
935      0             0        0       0       0              0
937      0             0        0       0       0              0
939      0             0        0       0       0              0

[400 rows x 6 columns]
0.6107227151130059
